## Load 20news data with python

The data is in form (group, word, frequency). This structure follows the COO sparse matrix format.

The last column (frequency) is not used in this project.

Finally print details to make sure loading data stats from here results the same values as provided by R example demo(0).

__R Demo(0) stats__:
- Number of groups: 20
- Number of words: 53975
- Number of documents: 11269
- Number of word-doc pairs: 1467345
- Density: 0.002412427

In [95]:
%load_ext autoreload
%autoreload 2

In [105]:
import pickle
import json
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, vstack
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

In [2]:
def load_20news_data(path_pfix="data/20news/"):
    X = []
    with open(f"{path_pfix}train.data", "r") as f:
        for i, line in enumerate(f.readlines()):
            line = line.split()
            assert len(line) == 3, f"line.split() returned incorrect number of elements: {len(line)} != 3"
            X.append(tuple(line[:2]))
    
    f = open(f"{path_pfix}train.label", "r")
    y = [label.strip() for label in f.readlines()]
    f.close()
    
    f = open(f"{path_pfix}vocabulary.txt", "r")
    vocab = [token.strip() for token in f.readlines()]
    f.close()
    print(X[0])
    return np.array(X, dtype=int), np.array(y, dtype=int), vocab


X, y, vocab = load_20news_data()
prev_docid = None
docid_count = 0
for x in X:
    if x[0] != prev_docid:
        docid_count += 1
        prev_docid = x[0]

assert docid_count == len(y), \
    "Number of doc_ids and labels should be equal, got {docid_count} doc_ids and {len(y)} labels."

groups = [
    'alt.atheism', 
    'comp.graphics',    
    'comp.os.ms-windows.misc', 
    'comp.sys.ibm.pc.hardware', 
    'comp.sys.mac.hardware', 
    'comp.windows.x', 
    'misc.forsale', 
    'rec.autos', 
    'rec.motorcycles', 
    'rec.sport.baseball', 
    'rec.sport.hockey', 
    'sci.crypt', 
    'sci.electronics', 
    'sci.med', 
    'sci.space', 
    'soc.religion.christian', 
    'talk.politics.guns',
    'talk.politics.mideast', 
    'talk.politics.misc', 
    'talk.religion.misc'
]


('1', '1')


In [3]:
print(f"• Number of groups: {len(groups)}")
print(f"• Number of words: {len(vocab)}")
print(f"• Number of documents: {docid_count}")
print(f"• Number of word-doc-pairs: {len(X)}")
print(f"• Density: {round(len(X)/(docid_count*len(vocab)), 9)}")

• Number of groups: 20
• Number of words: 53975
• Number of documents: 11269
• Number of word-doc-pairs: 1467345
• Density: 0.002412427


In [4]:
ynp = np.array(y, dtype=int)
group_index_ranges = []  # stores 2-tuples like (start_index, end_index) for each of the 20 groups
train_index_ranges = []
test_index_ranges = []
for i in range(1, 21):
    mask = (ynp == i)
    idx_range = np.where(mask)[0]
    split_idx = int((idx_range[-1]-idx_range[0])*0.9+idx_range[0])
    group_index_ranges.append((idx_range[0], idx_range[-1]))
    train_index_ranges.append((idx_range[0], split_idx))
    test_index_ranges.append((split_idx, idx_range[-1]+1))

In [5]:
train_index_ranges

[(0, 431),
 (480, 1002),
 (1061, 1574),
 (1633, 2160),
 (2220, 2736),
 (2795, 3326),
 (3387, 3909),
 (3969, 4500),
 (4561, 5096),
 (5157, 5690),
 (5751, 6288),
 (6349, 6882),
 (6943, 7474),
 (7534, 8067),
 (8128, 8660),
 (8721, 9259),
 (9320, 9809),
 (9865, 10371),
 (10429, 10845),
 (10893, 11230)]

In [6]:
test_index_ranges

[(431, 480),
 (1002, 1061),
 (1574, 1633),
 (2160, 2220),
 (2736, 2795),
 (3326, 3387),
 (3909, 3969),
 (4500, 4561),
 (5096, 5157),
 (5690, 5751),
 (6288, 6349),
 (6882, 6943),
 (7474, 7534),
 (8067, 8128),
 (8660, 8721),
 (9259, 9320),
 (9809, 9865),
 (10371, 10429),
 (10845, 10893),
 (11230, 11269)]

### Create conditional probability tables

"Estimate the probability that a document from the given group contains the word word."

Here we don't care about the word frequencies in documents, just the binary occurrence. Basically you need to count the number of documents in each group that has each word.

`p(w|g) = #(docs_having_word and docs_in_group) / #docs_in_group`

y-labels are in array thats length is the number of documents in the dataset. X on the other hand is currently in (implicit) sparse matrix format where first index indicates the document and second index the word. Construct a proper sparse matrix from X and concatenate Y to it.

The dataset was intended to be used with R and for that reason the indexing starts with 1 instead of 0. Remove the first row and first column from resulting sparse matrix

After the data in X is transformed, the documents can be directly aggregated based on group indexes given in y.

In [7]:
X_csr = csr_matrix((np.ones(X.shape[0]), (X[:, 0], X[:, 1])))
print(X_csr.shape)
print(X_csr[:, 0].sum() == 0)
print(X_csr[0, :].sum() == 0)
X_csr = X_csr[:, range(1, X_csr.shape[1])]  # remove first column as its all-zeroes
X_csr = X_csr[range(1, X_csr.shape[0]), :]  # remove first row as its all-zeroes
assert X_csr.sum() == X.shape[0], f"# of elements in sparse csr matrix does not match with the original"

(11270, 53976)
True
True


In [8]:
print(X_csr.shape)
print(X_csr.sum())
print(X_csr[:, 0].sum() == 0)
print(X_csr[0, :].sum() == 0)

(11269, 53975)
1467345.0
False
False


In [9]:
# so now to calculate p(w|g) we need to sum by row the subsets of X_csr defined by group indeces
# and then divide each element by # docs in group (length of group index range)
def empirical_posterior(X_csr, train_index_ranges):
    prob_stack = []
    for start, end in train_index_ranges:
        vec = X_csr[start:end, :].sum(axis=0)
        print(end-start)
        num_docs = end-start
        print(vec[0, :10])
        vec = vec/num_docs
        print(vec[0, :10])
        print(vec.ndim)
        print()
        prob_stack.append(vec.A.reshape(vec.shape[1]))

    prob_stack = np.array(prob_stack)
    return prob_stack


def laplace_smoothed_posterior(X_csr, train_index_ranges, alpha=1):
    """add-one smoothing when alpha=1"""
    prob_stack = []
    for start, end in train_index_ranges:
        vec = X_csr[start:end, :].sum(axis=0)
        print(end-start)
        num_docs = end-start
        print(vec[0, :10])
        vec = (vec+alpha)/(num_docs+alpha*X_csr.shape[1])
        print(vec[0, :10])
        print(vec.ndim)
        print()
        prob_stack.append(vec.A.reshape(vec.shape[1]))

    prob_stack = np.array(prob_stack)
    return prob_stack


prob_stack = laplace_smoothed_posterior(X_csr, train_index_ranges)

431
[[ 3. 33. 74.  4. 35. 30.  4.  1. 15. 48.]]
[[7.35213028e-05 6.24931074e-04 1.37852443e-03 9.19016285e-05
  6.61691725e-04 5.69790097e-04 9.19016285e-05 3.67606514e-05
  2.94085211e-04 9.00635959e-04]]
2

522
[[10. 34.  0.  7.  9. 39.  7.  5. 51.  3.]]
[[2.01845973e-04 6.42237187e-04 1.83496339e-05 1.46797071e-04
  1.83496339e-04 7.33985357e-04 1.46797071e-04 1.10097804e-04
  9.54180964e-04 7.33985357e-05]]
2

513
[[ 9. 39.  0. 10. 13. 32.  2.  2. 81.  0.]]
[[1.83526648e-04 7.34106592e-04 1.83526648e-05 2.01879313e-04
  2.56937307e-04 6.05637939e-04 5.50579944e-05 5.50579944e-05
  1.50491851e-03 1.83526648e-05]]
2

527
[[ 6. 18.  0.  0.  5. 41.  2.  1. 24.  0.]]
[[1.28435654e-04 3.48611060e-04 1.83479505e-05 1.83479505e-05
  1.10087703e-04 7.70613922e-04 5.50438516e-05 3.66959011e-05
  4.58698763e-04 1.83479505e-05]]
2

516
[[ 4. 19.  0.  1.  0. 35.  0.  0. 24.  0.]]
[[9.17582720e-05 3.67033088e-04 1.83516544e-05 3.67033088e-05
  1.83516544e-05 6.60659558e-04 1.83516544e-05 1.83516

In [10]:
print(prob_stack.shape)
print(prob_stack[0, :10])

(20, 53975)
[7.35213028e-05 6.24931074e-04 1.37852443e-03 9.19016285e-05
 6.61691725e-04 5.69790097e-04 9.19016285e-05 3.67606514e-05
 2.94085211e-04 9.00635959e-04]


prob_stack is a matrix shape (groups, vocab) that keeps the probabilities for number of documents having a word divided by number of documents in a group. Both empirical posterior and with Laplace smoothing has now been applied. About Laplace/Additive smoothing: https://en.wikipedia.org/wiki/Additive_smoothing

In [21]:
# here we will take a look at the most probable words. We assume that the most common words are common stopwords.

token_probabilities = prob_stack.sum(axis=0)
top_val_indeces = np.argsort(-token_probabilities)[:10]
vocab_entries_for_top_indeces = np.array(vocab)[top_val_indeces]
vocab_entries_for_top_indeces  # 10 most probable tokens in our vocab

array(['the', 'to', 'in', 'and', 'of', 'is', 'it', 'that', 'for', 'this'],
      dtype='<U79')

In [34]:
# now we will take a look at prior probabilities P(group) based on the populated group-vocab matrix
# the more densely-populated the group (more words in documents) the higher probabilities for tokens in vocab
# and once tokens are summed the scalar tells the relative prior.
# Records in prob_stack are already normalized by the number of documents in the set.
# Prob: relative prior divided by sum of all priors
group_priors = prob_stack.sum(axis=1)
normalizing_term = group_priors.sum()
group_priors = group_priors / normalizing_term
print(group_priors, group_priors.sum())
sorted_group_prior_indeces = np.argsort(-group_priors)
sorted_priors = np.sort(-group_priors)
group_prior_tuples = zip(np.array(groups)[sorted_group_prior_indeces], -sorted_priors)
print("\nGroup probabilities:")
list(group_prior_tuples)

[0.04912827 0.04468292 0.04295694 0.04432907 0.04182715 0.04870425
 0.03843853 0.04887348 0.04728229 0.04637036 0.0516335  0.06063393
 0.0458135  0.05409486 0.05375905 0.05988108 0.05684757 0.06585345
 0.05398022 0.04490958] 0.9999999999999998

Group probabilities:


[('talk.politics.mideast', 0.06585345013243205),
 ('sci.crypt', 0.06063392610148682),
 ('soc.religion.christian', 0.05988107553389996),
 ('talk.politics.guns', 0.05684757446077589),
 ('sci.med', 0.05409485886553029),
 ('talk.politics.misc', 0.05398022472323917),
 ('sci.space', 0.05375905481904335),
 ('rec.sport.hockey', 0.05163350093757131),
 ('alt.atheism', 0.049128268143443156),
 ('rec.autos', 0.04887348463781726),
 ('comp.windows.x', 0.048704249635702496),
 ('rec.motorcycles', 0.047282290318959724),
 ('rec.sport.baseball', 0.04637035644222927),
 ('sci.electronics', 0.04581349904539245),
 ('talk.religion.misc', 0.0449095802023898),
 ('comp.graphics', 0.044682917566578814),
 ('comp.sys.ibm.pc.hardware', 0.04432906815115269),
 ('comp.os.ms-windows.misc', 0.04295694258644816),
 ('comp.sys.mac.hardware', 0.041827145537828445),
 ('misc.forsale', 0.03843853215807878)]

## part 3

- the probability tables for feature to label mapping is already stored in prob_stack
- to get 'probabilities' (proportional to real probabilites) for each class, get product of features given class times prior of the class. 
- select largest value from previous step to be predicted class
- evaluate by plotting confusion matrix and misclassification error
- for both the training and test set

In [42]:
# prob_stack: prob distribution for tokens given group
# --> for each prob vector and corresponding group_priors (same index)

# construct a model holding these details (conditional probability vector for features, prior)
naive_bayes_model = {}  # access by key=group_class, vals=(k,v) for k=(feat_probs, prior), v=(f_vec, prior)

for i, row in enumerate(prob_stack):
    group_class = groups[i]
    g_prior = group_priors[i]
    print(group_class, g_prior, row.shape)
    naive_bayes_model[group_class] = {
        "feat_probs": row,
        "prior": g_prior
    }
    # now you have the group prior and conditional probability vector for features


alt.atheism 0.049128268143443156 (53975,)
comp.graphics 0.044682917566578814 (53975,)
comp.os.ms-windows.misc 0.04295694258644816 (53975,)
comp.sys.ibm.pc.hardware 0.04432906815115269 (53975,)
comp.sys.mac.hardware 0.041827145537828445 (53975,)
comp.windows.x 0.048704249635702496 (53975,)
misc.forsale 0.03843853215807878 (53975,)
rec.autos 0.04887348463781726 (53975,)
rec.motorcycles 0.047282290318959724 (53975,)
rec.sport.baseball 0.04637035644222927 (53975,)
rec.sport.hockey 0.05163350093757131 (53975,)
sci.crypt 0.06063392610148682 (53975,)
sci.electronics 0.04581349904539245 (53975,)
sci.med 0.05409485886553029 (53975,)
sci.space 0.05375905481904335 (53975,)
soc.religion.christian 0.05988107553389996 (53975,)
talk.politics.guns 0.05684757446077589 (53975,)
talk.politics.mideast 0.06585345013243205 (53975,)
talk.politics.misc 0.05398022472323917 (53975,)
talk.religion.misc 0.0449095802023898 (53975,)


In [101]:
# next we r gonna iterate each document in the training set and perform predictions using model from cell above.
# we select the argmax of classes in the model to be classification prediction y_hat
# all documents are in matrix X_csr.
# iterate each of the known 'training set indeces' and perform prediction
# store arrays y_true, y_pred each with dim (n, 20) where n is number of training samples


def predict_for_multiple_records(X, model):
    """
    The product of the feature probabilities tends to get very small which may cause numerical problems. 
    To avoid this, you can use a logarithmic scale to store the probabilities, i.e., initialize a 
    probability logp = log P (group) and add to it the term log P (wi | group) for all words wi occurring 
    in the document and the term log(1 − P (wi′ | group)) for all words wi′ that do not occur in the document. 
    The maximum probability class will have the highest logp value.
    """
    preds = []
    for i, row in enumerate(X):
        row = row.A.reshape(-1)  # convert into np array and reshape into 1d vector to match model 'coefs'
        # print(row.sum(), row.max(), row.min(), row.shape)
        cls_probs = []
        for cls, params in model.items():
            present_mask = (row == 1)
            absent_mask = (row == 0)
            assert (present_mask.sum()+absent_mask.sum()) == row.shape[0], "da fyck"
            
            tokens_present_vec = params["feat_probs"][present_mask]
            tokens_absent_vec = params["feat_probs"][absent_mask]
            tmp_ones = np.ones(tokens_absent_vec.shape[0])
            tokens_absent_vec = tmp_ones-tokens_absent_vec
            
            # transform feature probs into log probability to avoid numerical problems of extremely small numbers
            tokens_present_vec = np.log(tokens_present_vec)
            tokens_absent_vec = np.log(tokens_absent_vec)
        
            prob = params["prior"]+np.sum(tokens_present_vec)+np.sum(tokens_absent_vec)
            cls_probs.append(prob)
        
        preds.append(np.argmax(np.array(cls_probs)))
    return preds


def predictor(X_csr, index_ranges, naive_bayes_model):
    y_true = []
    y_pred = []
    i = 0
    for start, end in index_ranges:
        X_group = X_csr[start:end, :]
        group_preds = predict_for_multiple_records(X_group, naive_bayes_model)
        y_pred.extend(group_preds)
        y_true.extend([i]*X_group.shape[0])  # i=group_idx, repeated m times, m=number of train samples for group i
        i += 1
    return y_true, y_pred

In [104]:
y_true, y_pred = predictor(X_csr, train_index_ranges, naive_bayes_model)

In [106]:
# we have got all the training set predictions and labels now and we can analyze the results
accuracy = accuracy_score(y_true, y_pred)
miscls_error = 1-accuracy
print(groups)
print("\naccuracy:", accuracy)
print("miscls error:", miscls_error)

confusion = confusion_matrix(y_true, y_pred)
pd.DataFrame(confusion)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

accuracy: 0.8411945021259765
miscls error: 0.15880549787402354


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,296,0,0,0,0,0,0,0,0,0,0,1,0,0,0,106,0,28,0,0
1,0,362,4,13,1,17,2,1,0,0,1,91,0,1,4,20,1,4,0,0
2,1,4,381,19,0,21,0,0,0,0,0,70,0,2,1,10,0,3,1,0
3,0,2,0,433,0,3,2,0,0,0,0,66,0,1,1,9,3,7,0,0
4,0,3,1,12,346,3,0,1,1,0,0,122,3,3,0,16,2,3,0,0
5,0,1,3,1,0,492,0,0,0,0,0,25,0,0,0,4,1,3,1,0
6,0,0,2,35,3,0,289,17,1,1,4,82,11,4,7,18,11,37,0,0
7,0,0,0,0,0,2,0,485,0,1,0,10,0,0,0,8,7,17,1,0
8,0,1,0,0,0,0,4,1,492,0,0,6,0,1,0,6,9,15,0,0
9,0,0,0,0,0,2,0,1,0,487,8,5,0,2,1,9,4,14,0,0


In [102]:
y_true, y_pred = predictor(X_csr, test_index_ranges, naive_bayes_model)

In [103]:
# now we have test set results, lets present them below
accuracy = accuracy_score(y_true, y_pred)
miscls_error = 1-accuracy
print(groups)
print("\naccuracy:", accuracy)
print("miscls error:", miscls_error)

confusion = confusion_matrix(y_true, y_pred)
pd.DataFrame(confusion)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

accuracy: 0.6461937716262975
miscls error: 0.35380622837370246


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,14,0,0,0,0,0,0,0,0,0,0,2,0,0,0,27,0,6,0,0
1,0,25,0,2,0,9,0,0,0,0,0,19,0,1,0,2,0,1,0,0
2,0,1,16,5,0,7,0,1,0,0,0,25,0,0,1,0,2,1,0,0
3,0,0,2,36,1,1,2,0,0,0,0,13,1,0,0,3,1,0,0,0
4,0,1,2,3,23,1,0,0,0,0,1,24,1,1,0,1,1,0,0,0
5,0,2,0,0,0,47,0,0,0,0,0,9,0,0,2,0,0,1,0,0
6,0,0,0,5,0,1,19,7,1,0,0,15,3,0,1,3,4,1,0,0
7,0,0,0,0,0,0,1,43,0,0,0,3,0,0,2,4,4,4,0,0
8,0,0,0,0,0,0,0,1,45,0,0,1,1,0,0,5,1,7,0,0
9,0,0,0,0,0,0,0,0,0,41,7,1,0,0,0,2,5,5,0,0
